# <div align="center"> k12学习框架错误码 </div>

## 一. 统一错误码

In [1]:
from IPython.display import Markdown
from k12libs.common.k12ai_errmsg import ERRORS

In [2]:
table = '''
| 错误码(code) | 英文描述(message.en) | 中文描述(message.cn) |
| :---: | :--- | :--- |
|<img width=200/> | <img width=300/> | <img width=300/> |
'''
for errcode, desc in ERRORS.items():
    desc_en = desc['en']
    desc_cn = desc['cn']
    table += f'| {errcode} | {desc_en} | {desc_cn} |\n| |\n'
    
Markdown(table)


| 错误码(code) | 英文描述(message.en) | 中文描述(message.cn) |
| :---: | :--- | :--- |
|<img width=200/> | <img width=300/> | <img width=300/> |
| 100000 | success | 成功 |
| |
| 100101 | api parameter key is not found | API参数错误: 非法Key |
| |
| 100102 | api parameter value is invalid | API参数错误: 非法Value |
| |
| 100103 | api parameter json format error | API参数错误: 非法Json格式 |
| |
| 100199 | api parameter unknown exception | API参数错误: Unknow异常 |
| |
| 100201 | task service is not found | 任务启动错误: 请求的服务不存在或者服务未启动 |
| |
| 100202 | task start fail | 任务启动错误: 启动失败 |
| |
| 100203 | task start docker fail | 任务启动错误: 容器启动失败 |
| |
| 100204 | task is running | 任务启动错误: 服务已经启动,正在运行 |
| |
| 100205 | task is not found or not running | 任务停止错误: 请求的服务不存在或者未启动 |
| |
| 100206 | task schema file is not found | 模板请求错误: 模板文件不存在 |
| |
| 100207 | task schema get fail | 模板请求错误: 模板获取失败(内部错误) |
| |
| 100208 | task evaluate has no model file | 任务启动错误: 评估服务需要在完成训练后执行 |
| |
| 100209 | task evaluate has no config file | 任务启动错误: 评估服务缺少配置文件 |
| |
| 100231 | task parameter is invalid | 任务启动错误: 非法服务参数 |
| |
| 100232 | task parameter has no key: input file | 任务启动错误: 评估服务缺少输入文件 |
| |
| 100233 | task parameter key is missing | 任务启动错误: 缺少服务参数 |
| |
| 100299 | task unknown exception | 任务启动错误: Unkown异常 |
| |
| 100302 | cv model name is not valid | 框架运行错误: CV框架非法模型 |
| |
| 100303 | cv optimizer method is not valid | 框架运行错误: CV框架非法优化方法 |
| |
| 100304 | cv pad mode is not valid | 框架运行错误: CV框架非法填充模式 |
| |
| 100305 | cv anchor method is not valid | 框架运行错误: CV框架非法Anchor方式 |
| |
| 100306 | cv image format type is not valid | 框架运行错误: CV框架非法图片类型 |
| |
| 100307 | cv tensor size error | 框架运行错误: CV框架Tensor大小不合理 |
| |
| 100901 | out of memory | 常见错误: 内存溢出 |
| |
| 100902 | not implemented yet | 常见错误: 尚未实现 |
| |
| 100903 | configuration error | 常见错误: 参数配置错误 |
| |
| 100904 | missing key configuration | 常见错误: 配置中缺少参数 |
| |
| 100905 | file is not found | 常见错误: 文件不存在 |
| |
| 999999 | unknown error! | 常见错误: 不可知错误 |
| |


## 二. 错误码模型定义

```json
{
    "code": 101,  // [M] 统一错误码
    "descr": "",  // [M] 错误简要信息
    "message":"", // [O] 自定义消息,辅助消息
    "data": "",   // [O] 携带消息
    "detail": {     // [O] 错误详细信息
        "filename": "", // [C] 非异常错误时, 错误输出所在文件
        "linenum": 100, // [C] 非异常错误时, 错误输出所在行号
        "funcname": "", // [C] 非异常错误时, 错误输出所在函数
        "err_type": "", // [C] 错误是异常时, 异常的类名
        "err_text": "", // [C] 错误是异常时, 异常的内容
        "traceback":[   // [C] 错误是异常时, 错误堆栈
            {
                "filename": "",
                "linenum": 100,
                "funcname": "",
                "source": "" // [C] 错误是异常时, 发生问题的程序代码
            },
            {
                "filename": "",
                "linenum": 100,
                "funcname": "",
                "source": "" // [C] 错误是异常时, 发生问题的程序代码
            },
        ]
    }
}
```